In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from statsmodels.graphics import tsaplots
import statsmodels.api as sm

In [ ]:
plt.style.available

In [ ]:
# plt.rcParams.keys()

In [ ]:
# sns.heatmap(correlation_matrix,
#             annot=True,
#             linewidths=0.4,
#             annot_kws={"size": 10})

### import data

In [ ]:
appointments = pd.read_csv('Appointments.csv')

In [ ]:
calls = pd.read_csv('CallsRingCentral.csv')

In [ ]:
reason_for_visit = pd.read_csv('MeetingReasonForVisits.csv')

In [ ]:
meeting_status = pd.read_csv('MeetingStatus.csv')

In [ ]:
offices = pd.read_csv('Offices.csv')

In [ ]:
providers_schedules = pd.read_csv('ProvidersSchedulesLastest.csv')

### explore data

In [ ]:
appointments.info()

In [ ]:
appointments.head().T

In [ ]:
reason_for_visit.head()

In [ ]:
reason_for_visit.info()

In [ ]:
len(reason_for_visit['Name'].unique())

In [ ]:
meeting_status.head()

In [ ]:
meeting_status.info()

In [ ]:
offices.head()

In [ ]:
offices.info()

In [ ]:
providers_schedules.head()

In [ ]:
providers_schedules.info()

In [ ]:
calls.head().T

In [ ]:
calls.info()

### combine/merge dataframes

In [ ]:
appointments['Specialty'].unique()

In [ ]:
pd.value_counts(appointments['Specialty'])

In [ ]:
doctors = ['Psychiatry', 'Child & Adolescent Psychiatry', ]
RN_PAs = ['Medical', 'Psych/Mental Health, Child & Adolescent', 'Psych/Mental Health', 'Physician Assistant']
therapists = ['Marriage & Family Therapist', 'Psychologist', 'Specialist/Technologist, Other', 'Clinical' ]

In [ ]:
appointments['Specialty'].loc[appointments['Specialty'].isin(doctors)]= 'doctor'
appointments['Specialty'].loc[appointments['Specialty'].isin(RN_PAs)] = 'RN/PA'
appointments['Specialty'].loc[appointments['Specialty'].isin(therapists)] = 'therapist'

In [ ]:
pd.value_counts(appointments['Specialty'])

In [ ]:
merged1 = pd.merge(left=appointments, right=reason_for_visit, how='left', left_on='MeetingReasonForVisitId',\
                  right_on='Id')

In [ ]:
merged1 = merged1.rename(columns={'MeetingReasonForVisitId': 'ReasonForVisitId', 'Name':'ReasonForVisitName', 'Description':'ReasonForVisitDescription'})

In [ ]:
merged1.columns

In [ ]:
merged1.drop('Id', axis=1, inplace=True)

In [ ]:
merged1.head().T

In [ ]:
# merge in office name from offices df
merged1 = pd.merge(left=merged1, right=offices, how='left', left_on='OfficeId', right_on='id')

In [ ]:
merged1 = merged1.rename(columns={'Name':'OfficeName', 'id_x':'id'})

In [ ]:
merged1.drop('id_y', axis=1, inplace=True)

In [ ]:
merged1 = pd.merge(left=merged1, right=meeting_status, how='left', left_on='MeetingStatusId', right_on='Id')

In [ ]:
merged1 = merged1.rename(columns={'Name':'MeetingStatusName', 'Description':'MeetingStatusDescription'})

In [ ]:
merged1.drop('Id', axis=1, inplace=True)

In [ ]:
merged1.info()

In [ ]:
merged1.head().T

In [ ]:
# rearrange column order to group releveant columns together
merged1.columns

In [ ]:
ordered_columns = ['id', 'Patient', 'PatientAgeMeetingDate', 'PatientGender',
       'PatientState', 'PatientCity', 'PatientInsurance', 'Provider',
       'Specialty', 'AppointmentDate', 'AppointmentCreated', 'AppointmentDuration', 'ReasonForVisitId', 'ReasonForVisitName',
       'ReasonForVisitDescription','MeetingStatusId', 'MeetingStatusName',
       'MeetingStatusDescription', 'OfficeId',  'OfficeName', 'CreatedBy']

In [ ]:
merged1 = merged1[ordered_columns]

In [ ]:
# id any missing specialties
merged1['Specialty'].isnull().sum(), merged1['Specialty'].notnull().sum()

### Data Cleaning: 
#### filling NaN values

In [ ]:
no_specialty = merged1[appointments['Specialty'].isnull()] 

In [ ]:
no_specialty = no_specialty[['Provider', 'Specialty', 'AppointmentDate', 'AppointmentCreated',\
        'AppointmentDuration', 'ReasonForVisitId', 'ReasonForVisitName',
       'ReasonForVisitDescription','MeetingStatusId', 'MeetingStatusName', 'MeetingStatusDescription', \
    'OfficeId',  'OfficeName']]

In [ ]:
no_specialty.head().T

In [ ]:
pd.value_counts(no_specialty['ReasonForVisitName'])

In [ ]:
implied_therapy = ['Therapy', 'New Patient Therapy', ]
implied_doctor = ['Therapy Telepsychiatry','Follow up Telepsychiatry', 'New Patient Therapy Telepsychiatry',\
                  'New Patient MD Adult', 'New Patient MD Adult Telepsychiatry']
merged1['Specialty'].loc[merged1['ReasonForVisitName'].isin(implied_therapy)] = 'therapist'
merged1['Specialty'].loc[merged1['ReasonForVisitName'].isin(implied_doctor)] = 'doctor'


In [ ]:
# most missing values in Specialty are now filled
merged1['Specialty'].isnull().sum(), merged1['Specialty'].notnull().sum()

In [ ]:
# # remove time component from AppointmentDate and AppointmentCreated columns
# merged1['AppointmentCreated'] = merged1['AppointmentCreated'].str.slice(start=0, stop=10)
# merged1['AppointmentDate'] = merged1['AppointmentDate'].str.slice(start=0, stop=10)

In [ ]:
# pd.value_counts(merged1['PatientState']), pd.value_counts(merged1['PatientCity'])

In [ ]:
# pd.value_counts(merged1['PatientGender']), # pd.value_counts(merged1['MeetingStatusName'])

In [ ]:
pd.value_counts(merged1['AppointmentDuration'])

In [ ]:
pd.value_counts(merged1[merged1['AppointmentDuration'] > 90]['ReasonForVisitName'])

In [ ]:
# convert date columns to datetime 
# merged1['AppointmentDate'] = pd.to_datetime(merged1['AppointmentDate'].apply(lambda x: x.date()) #,format='%Y-%m-%d')
# merged1['AppointmentDate'] = pd.to_datetime(merged1['AppointmentDate'].apply(lambda x: x.date()) #,format='%Y-%m-%d')

In [ ]:
merged1['AppointmentCreated'] = pd.to_datetime(merged1['AppointmentCreated'], errors='coerce')#.apply(lambda x: x.date()) #, format='%Y-%m-%d')
merged1['AppointmentDate'] = pd.to_datetime(merged1['AppointmentDate'], errors='coerce')#.apply(lambda x: x.date()) #, format='%Y-%m-%d')

In [ ]:
# calculate time between AppointmentCreated and AppointmentDate
merged1['DaysFromAppointmentCreatedToVisit'] = (merged1['AppointmentDate'] - merged1['AppointmentCreated']).dt.days

In [ ]:
merged1.info()

In [ ]:
# merged1['Specialty'].isnull()
merged1.isnull().sum()

In [ ]:
# pd.value_counts(merged1['DaysFromAppointmentCreatedToVisit'])

In [ ]:
merged1[merged1['DaysFromAppointmentCreatedToVisit'] < 0][['DaysFromAppointmentCreatedToVisit', 'AppointmentCreated', 'AppointmentDate']]

In [ ]:
# merged1['AppointmentDate'] = pd.to_datetime(merged1['AppointmentDate'])
# merged1['AppointmentCreated'] = pd.to_datetime(merged1['AppointmentCreated'])

In [ ]:
# merged1.dtypes

In [ ]:
pd.value_counts(merged1['Specialty'])

In [ ]:
merged2 = merged1.set_index('AppointmentDate')

In [ ]:
merged_index_month = merged2.index.month

In [ ]:
merged_index_year = merged2.index.year

In [ ]:
merged2.index.date

In [ ]:
merged2

In [ ]:
specialty_counts = merged2.groupby([merged_index_year, merged_index_month, 'Specialty'])['Specialty'].count()

In [ ]:
specialty_counts.head()

In [ ]:
specialty = merged2[['Specialty']].dropna(axis=0)

In [ ]:
specialty.isnull().sum()

In [ ]:
type(specialty)

In [ ]:
a_index_year = specialty.index.year
a_index_month = specialty.index.month

In [ ]:
specialty_bfilled = specialty.fillna(method='bfill')

In [ ]:
cols = list(specialty.columns)
means = specialty[cols].mean()
means.index

In [ ]:
# fill missing values with column mean
for idx in means.index:
    specialty_df[idx].fillna(means[idx], inplace=True)

In [ ]:
# plot number of appoints by specialty for each month, adding multiple years
ax3 = specialty.groupby([a_index_year, a_index_month, 'Specialty'])['Specialty'].count().unstack(\
                            'Specialty').plot(figsize=(8,8),colormap='Dark2', fontsize=12)
ax3.set_ylim(0, 2000)
ax3.set_xlim(0,28)
ax3.set_title('Number of Appointments per Month by Specialty', fontsize=15)
ax3.set_xlabel('Date', fontsize=12)
ax3.set_ylabel('Number of Appointments', fontsize=12)
ax3.legend(loc='center left', bbox_to_anchor=(1.0, 0.5), fontsize=20)
plt.show()

In [ ]:
# plot number of appoints by specialty for each month, adding multiple years
ax2 = specialty.groupby([a_index_year, a_index_month, 'Specialty'])['Specialty'].count().unstack(\
            'Specialty').plot.area(figsize=(8,8), colormap='Dark2', fontsize=12)
                                                    
ax2.set_ylim(0, 3000)
ax2.set_xlim(0,28)
ax2.set_title('Number of Appointments per Month by Specialty', fontsize=15)
ax2.set_xlabel('Date', fontsize=12)
ax2.set_ylabel('Number of Appointments', fontsize=12)
ax2.legend(loc='center left', bbox_to_anchor=(1.0, 0.5), fontsize=12)
plt.show()

In [ ]:
# # plot number of appoints by specialty for each month, adding months from multiple years together
# ax = merged2.groupby([merged_index_month, 'Specialty'])['Specialty'].count().unstack().plot(figsize=(8,8),\
#                                                     colormap='Dark2')
# ax.set_ylim(0, 4000)
# ax.set_xlim(0,13)
# ax.set_title('Summed Number of Appointments by Month of Year', fontsize=15)
# ax.set_xlabel('Month', fontsize=12)
# ax.set_ylabel('Number of Appointments', fontsize=12)
# ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5), fontsize=12)
# plt.show()

In [ ]:
# # plot number of appoints by specialty for each month, adding months from multiple years together
# ax1 = merged2.groupby([merged_index_month, 'Specialty'])['Specialty'].count().unstack().plot.area(figsize=(8,8),\
#                                                                                                 colormap='Accent')
# ax1.set_ylim(0, 5000)
# ax1.set_xlim(1, 12)
# ax1.set_title('Summed Number of Appointments by Month of Year', fontsize=18)
# ax1.set_xlabel('Month', fontsize=12)
# ax1.set_ylabel('Number of Appointments', fontsize=12)
# ax1.legend(loc='center left', bbox_to_anchor=(1.0, 0.5), fontsize=12)
# plt.show()

In [ ]:
# ax = merged2.groupby([merged_index_month, 'Specialty'])['Specialty'].count().unstack().plot(subplots=True,\
#     colormap='Dark2', figsize=(10,8), layout=(3,1), sharex=True, sharey=False, fontsize=10, linewidth=4)
# plt.xlabel('Month', fontsize=10)
# plt.title('Summed Number of Appointments by Month of Year', fontsize=15)

# plt.tight_layout()
# plt.show()

In [ ]:
# merged2.head(2).T

In [ ]:
merged2['DurationHours'] = merged2['AppointmentDuration'] /60

In [ ]:
time_spent_by_month = merged2.groupby([merged_index_year, merged_index_month, 'Specialty'])['DurationHours'].sum()

In [ ]:
time_spent_by_month.head()

In [ ]:
merged2.groupby([merged_index_year, merged_index_month])['DurationHours'].sum().plot(figsize=(10,6))
plt.xlabel('Date')
plt.ylabel('Time (hours)')
plt.title('Sum of time spent for all Specialties together')
plt.show()

In [ ]:
ax6 = merged2.groupby([merged_index_year, merged_index_month, 'Specialty'])['DurationHours']\
.sum().unstack().plot(figsize=(10,8),colormap='Dark2', linewidth=3, fontsize=12)
                                                    
ax6.set_ylim(0, 2000)
ax6.set_xlim(0,25)
ax6.set_title('Time Spent by Specialty (hours)', fontsize=16)
ax6.set_xlabel('Date', fontsize=15)
ax6.set_ylabel('Time (minutes)', fontsize=15)
ax6.legend(loc='center left', bbox_to_anchor=(1.0, 0.5), fontsize=12)
plt.show()

In [ ]:
# Area plot of time spent for each specialty
ax7 = merged2.groupby([merged_index_year, merged_index_month, 'Specialty'])['DurationHours']\
.sum().unstack().plot.area(figsize=(10,8), colormap='Dark2', linewidth=3, fontsize=10)
                                                   
ax7.set_ylim(0, 2500)
ax7.set_xlim(0,25)
ax7.set_title('Time Spent by Specialty (hours)', fontsize=16)
ax7.set_xlabel('Date', fontsize=15)
ax7.set_ylabel('Time (minutes)', fontsize=15)
ax7.legend(loc='center left', bbox_to_anchor=(1.0, 0.5), fontsize=12)
plt.show()

In [ ]:
date = merged2.index
merged2.groupby([date,'Specialty'])['DurationHours'].sum()

In [ ]:
date = pd.to_datetime(merged2.index,format='%Y-%m-%d')

In [ ]:
merged2['AppointmentDate'] = date

In [ ]:
# merged2.head().T

In [ ]:
type(merged2['AppointmentDate'])

In [ ]:
merged2.groupby(['AppointmentDate', 'Specialty'])['DurationHours'].sum()

In [ ]:
# date = pd.to_datetime(merged2.index)
ax8 = merged2.groupby(['AppointmentDate', 'Specialty'])['DurationHours'].sum().unstack().\
plot.area(figsize=(8,8), colormap='Dark2', linewidth=3, fontsize=10)
                                                    
# ax8.set_ylim(0, 32000)
# ax8.set_xlim(0,25)
ax8.set_title('Time per Month by Specialty (hours)', fontsize=18)
ax8.set_xlabel('Date', fontsize=15)
ax8.set_ylabel('Time (minutes)', fontsize=15)
ax8.legend(loc='center left', bbox_to_anchor=(1.0, 0.5), fontsize=12)
plt.show()

In [ ]:
params = {'legend.fontsize': 20, 'legend.handlelength': 2, 'axes.labelsize': 'medium'}
plt.rcParams.update(params)
fig = merged2.groupby(['AppointmentDate', 'Specialty'])['DurationHours'].sum().unstack().plot.area(subplots=True,\
    colormap='Dark2', figsize=(20,50), layout=(10,1), sharex=True, sharey=True, linewidth=3, fontsize=20)
plt.xlabel('Date')
plt.ylabel('Time (hours)', fontsize=20)
plt.tight_layout()
plt.show()

In [ ]:
merged2.columns

In [ ]:
merged2['AppointmentDate'] = merged2.index

In [ ]:
duration_df = merged2[['Provider', 'Specialty', 'AppointmentCreated', 'AppointmentDate', 'AppointmentDuration',
       'ReasonForVisitName', 'DurationHours', 'ReasonForVisitDescription','MeetingStatusName', 'MeetingStatusDescription',
       'OfficeId']]

In [ ]:
duration_df.info()

In [ ]:
duration_df.isnull().sum()

In [ ]:
# drop appointments that are longer than 90 minutes
duration_df = duration_df[duration_df['AppointmentDuration'] <= 90]

In [ ]:
# drop remaining columns with missing values
duration_df.dropna(axis=0, inplace=True)

In [ ]:
# params = {'legend.fontsize': 20, 'legend.handlelength': 2, 'axes.labelsize': 'medium'}
# plt.rcParams.update(params)
# fig2 = duration_df.groupby(['AppointmentDate', 'Specialty'])['AppointmentDuration'].sum().unstack().plot.area(\
#     colormap='Dark2', figsize=(40,20), linewidth=3, fontsize=20)
# plt.xlabel('Date')
# plt.ylabel('Time (minutes)', fontsize=20)
# plt.tight_layout()
# plt.show()

In [ ]:
params = {'legend.fontsize': 20, 'legend.handlelength': 2, 'axes.labelsize': 'medium'}
plt.rcParams.update(params)
fig2 = duration_df.groupby(['AppointmentDate', 'Specialty'])['DurationHours'].sum().unstack().plot.area(subplots=True,\
    colormap='Dark2', figsize=(20,60), layout=(10,1), sharex=True, sharey=True, linewidth=3, fontsize=20)
plt.xlabel('Date')
plt.ylabel('Time (minutes)', fontsize=20)
plt.tight_layout()
plt.show()

In [ ]:
doctors = duration_df[duration_df['Specialty'] == 'doctor']
therapists = duration_df[duration_df['Specialty'] == 'therapist']
RN_PA = duration_df[duration_df['Specialty'] == 'RN/PA']

In [ ]:
doc_fig = doctors.groupby('AppointmentDate')['DurationHours'].sum().plot.area(figsize=(12,10), \
        fontsize=10)                      
plt.title('Doctors', fontsize=15)
plt.xlabel('Date')
plt.ylabel('Time (hours)', fontsize=12)
plt.show()

In [ ]:
doc_duration = doctors.groupby(doctors.index.date)['DurationHours'].sum()
RN_PA_duration = RN_PA.groupby(RN_PA.index.date)['DurationHours'].sum()
therapist_duration = therapists.groupby(therapists.index.date)['DurationHours'].sum()

In [ ]:
ax = doc_duration.plot(figsize=(10,6), linewidth = 3, fontsize=10, grid=True)
ax.set_title('Doctors Time Spent', fontsize=18)
ax.set_xlabel('Date', fontsize=15)
ax.set_ylabel('Time (hours)', fontsize=15)
plt.show()

In [ ]:
# take first difference
params = {'figure.figsize': [16,6],'axes.labelsize': 'medium', 'font.size': 10.0, 'lines.linewidth': 2}
plt.rcParams.update(params)
plt.plot(doc_duration.diff())
plt.show()

In [ ]:
# adjust for seasonal effect by taking 4th difference
params = {'figure.figsize': [16,6],'axes.labelsize': 'medium', 'font.size': 10.0, 'lines.linewidth': 2}
plt.rcParams.update(params)
plt.plot(doc_duration.diff(4))
plt.show()

In [ ]:
# downsample from daily to weekly & montly data
weekly_doc_dur = doc_duration.resample(rule='W').last() # weekly time spent

ax = weekly_doc_dur.plot(figsize=(10,6), linewidth = 3, fontsize=10, grid=True)
ax.set_title('Doctors Time Spent Weekly', fontsize=18)
ax.set_xlabel('Date', fontsize=15)
ax.set_ylabel('Time (hours)', fontsize=15)
plt.show()

In [ ]:
# How should resampling be done: last, first, mean????
monthly_doc_dur = doc_duration.resample(rule='M').last() # monthly time spent
ax = monthly_doc_dur.plot(figsize=(10,6), linewidth = 3, fontsize=10, grid=True)
ax.set_title('Doctors Time Spent Monthly', fontsize=18)
ax.set_xlabel('Date', fontsize=15)
ax.set_ylabel('Time (hours)', fontsize=15)
plt.show()

In [ ]:
ax = RN_PA_duration.plot(figsize=(10,6), linewidth = 3, fontsize=10, grid=True)
ax.set_title('RN_PA Time Spent', fontsize=18)
ax.set_xlabel('Date', fontsize=15)
ax.set_ylabel('Time (hours)', fontsize=15)
plt.show()

In [ ]:
ax = therapist_duration.plot(figsize=(10,6), linewidth = 3, fontsize=10, grid=True)
ax.set_title('Therapists Time Spent', fontsize=18)
ax.set_xlabel('Date', fontsize=15)
ax.set_ylabel('Time (hours)', fontsize=15)
plt.show()

In [ ]:
# Plot moving average
doc_mean = pd.rolling_mean(doc_duration, window=30)
ax = doc_mean.plot(figsize=(10,6), linewidth=2, fontsize=12)
ax.set_title('60 day rolling mean of doc duration', fontsize=18)
ax.set_xlabel('Date', fontsize=15)
plt.show()

In [ ]:
plt.rcParams["figure.figsize"] = [8,8]
doc_acf = tsaplots.plot_acf(doc_duration, lags=31, alpha=0.05)
plt.title('Doctors Autocorrelation', fontsize=20)
plt.show()

In [ ]:
plt.rcParams["figure.figsize"] = [8,8]
doc_pacf = tsaplots.plot_pacf(doc_duration, lags=31, alpha=0.05)
plt.title('Doctors Partial Autocorrelation', fontsize=10)
plt.show()

In [ ]:
d_ts_index = pd.to_datetime(doc_duration.index)
RN_ts_index = pd.to_datetime(RN_PA_duration.index)
t_ts_index = pd.to_datetime(therapist_duration.index)

In [ ]:
doc_duration.index = d_ts_index
RN_PA_duration.index = RN_ts_index
therapist_duration.index = t_ts_index

In [ ]:
params = {'figure.figsize': [20.0, 20.0],'axes.labelsize': 'large', 'font.size': 20.0, 'lines.linewidth': 3}
plt.rcParams.update(params)
doc_decomp = sm.tsa.seasonal_decompose(doc_duration, freq=30)
fig = doc_decomp.plot()
plt.title('Doctors')
plt.show()

In [ ]:
params = {'figure.figsize': [20.0, 20.0],'axes.labelsize': 'large', 'font.size': 20.0, 'lines.linewidth': 3}
plt.rcParams.update(params)
RN_PA_decomp = sm.tsa.seasonal_decompose(RN_PA_duration, freq=30)
fig = RN_PA_decomp.plot()
plt.title('RN_PA')
plt.show()

In [ ]:
params = {'figure.figsize': [20.0, 20.0],'axes.labelsize': 'large', 'font.size': 20.0, 'lines.linewidth': 3}
plt.rcParams.update(params)
therapist_decomp = sm.tsa.seasonal_decompose(therapist_duration, freq=30)
fig = therapist_decomp.plot()
plt.title('Therapists')
plt.show()

In [ ]:
plt.rcParams.keys()

In [ ]:
# index_year = df.index.year
# df_by_year = df.groupby(index_year).mean()